### PySpark Library

In [ ]:
%pyspark
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
from pyspark.sql.types import (StructField,FloatType,TimestampType,StructType)
from pyspark.sql import functions as f
from pyspark.sql.functions import format_number

## Job Start Time

In [ ]:
%pyspark
import time
start_time = time.time()

## Import Data

In [ ]:
%pyspark
df = spark.read.csv("s3://aws-logs-664959780319-us-east-1/data/10lkh.csv",header=True)
df.printSchema()
df.show(10)

## Compute Log Returns

In [ ]:
%pyspark
df3 = df
df3 = df3.withColumn("Id", lit('1'))
w = Window().partitionBy().orderBy(col("Id"))
df3 = df3.select("*", lag("close").over(w).alias("close_shift"))
df3 = df3.withColumn('returns', pyspark.sql.functions.log(col('close') / col('close_shift'))*100)
df3.show()

## Descriptive Statistics

In [ ]:
%pyspark
result = df3.describe()
result.select(result['summary'],format_number(result['returns'].cast('float'),6).alias('returns'),).show()
df3.agg(f.skewness(df3.returns)).show()
df3.agg(f.kurtosis(df3.returns)).show()

## Job End - Decision Latency

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))